# Введение

In [ ]:
import warnings
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes, fetch_openml,load_iris,fetch_california_housing
from sklearn.feature_selection import mutual_info_regression, f_regression, RFE, SelectFromModel, SelectKBest, f_classif
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import (
RepeatedStratifiedKFold, 
cross_val_score, 
train_test_split, 
GridSearchCV,
cross_val_predict, 
learning_curve, 
validation_curve)
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error,zero_one_loss, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.decomposition import PCA
from sklearn.datasets import make_circles, make_moons, make_blobs
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler
from os.path import join as pjoin
from mlxtend.evaluate import bias_variance_decomp
#sharper plots
%config InlineBackend.figure_format = 'retina'

from sklearn.linear_model import (LogisticRegression, LogisticRegressionCV,
                                  SGDClassifier)

warnings.filterwarnings("ignore")

In [ ]:
data_path = "D:/data/ml" 

## Проклятие размерности

 Проклятие размерности означает, что по мере увеличения количества объектов или измерений в наборе данных объем данных, необходимый для эффективного моделирования взаимосвязи между объектами и целевой переменной, растет экспоненциально.
Чем больше мы добавляем признаков, не увеличивая объем данных, используемых для обучения модели, тем больше увеличивается среднее расстояние между точками в пространстве признаков. Из-за такой разреженности становится гораздо проще найти удобное и совершенное, но не столь оптимальное решение для модели машинного обучения. Следовательно, модель плохо обобщает, что делает прогнозы ненадежными. Это может привести к таким проблемам, как переобучение и снижение точности.

 Чтобы смягчить проклятие размерности, можно использовать такие методы, как выбор признаков, уменьшение размерности и ансамблевые методы.

 Для примера рассмотрим 1000 случайных точек в пространствах размерности от 2 до 50. Построим график, показывающий это свойство.

In [ ]:
deltas = []
for N in range(2,50):
    # Generate 1000 random points in N dimensions.
    P = [np.random.randint(-100, 100, N) for _ in range(10000)]
    Q = # YOUR CODE : generate random point
    diffs = [np.linalg.norm(p-Q) for p in P]
    mxd = max(diffs)
    mnd = min(diffs)
    delta = math.log10(mxd-mnd)/mnd # this shows the tightness of points region
    deltas.append( delta )

plt.plot(range(2,50),deltas)
plt.xlabel('Number of dimensions')
plt.ylabel('Width of points region')
plt.show()

In [ ]:
deltas = []
for N in range(2,50):
    # Generate 1000 random points in N dimensions.
    P = [np.random.randint(-100, 100, N) for _ in range(10000)]
    Q = np.random.randint(-100,100,N)
    diffs = [np.linalg.norm(p-Q) for p in P]
    mnd = min(diffs)
    delta = # YOUR CODE: plot logarithm of mean distance
    deltas.append( delta )

plt.plot(range(2,50),deltas)
plt.xlabel('Number of dimensions')
plt.ylabel('Mean Euclidean Distance')
plt.show()

Для того, чтобы уменьшить влияние проклятия размерности, используются разные техники:
 - Feature selection. Выбор наиболее релевантных функций из набора данных может помочь уменьшить размерность и повысить производительность модели.
 - Снижение размерности. Такие методы, как анализ главных компонент (PCA), линейный дискриминантный анализ (LDA) и t-SNE, могут использоваться для уменьшения размерности данных при сохранении наиболее важной информации.
 - Регуляризация. Методы регуляризации, такие как L1 и L2, могут помочь предотвратить переобучение в многомерных наборах данных путем добавления штрафного члена к функции потерь.
 - Ансамблевые методы. Объединение результатов нескольких моделей также может помочь повысить производительность в многомерных наборах данных.
 - Увеличение размера набора обучающих данных также может помочь преодолеть проклятие размерности, однако сбор большего количества данных может занять много времени и стоит дорого.
 - Генерация синтетических данных. Генерацию синтетических данных также можно использовать для увеличения размера набора данных, особенно когда сбор большего количества реальных данных невозможен.

Рассмотрим несколько вариантов решений.

### Feature selection

UCI MAGIC dataset: https://archive.ics.uci.edu/ml/datasets/magic+gamma+telescope

MAGIC (Major Atmospheric Gamma Imaging Cherenkov) - это система, состоящая из двух черенковских телескопов диаметром 17 м. Они предназначены для наблюдения гамма-лучей от галактических и внегалактических источников в диапазоне очень высоких энергий (от 30 ГэВ до 100 ТэВ).

Телескопами MAGIC в настоящее время управляют около 165 астрофизиков из 24 организаций и консорциумов из 12 стран. MAGIC позволил открыть и исследовать новые классы источников гамма-излучения, таких как, например, пульсары и гамма-всплески (GRB).

В наборе данных хранятся параметры зарегистрированных "фотографий" излучений потоков частиц от этих телескопов. Они делятся на  кванты (фотоны) и адроны (протоны). 
Описание признаков:

- Length: continuous # major axis of ellipse [mm]
- Width: continuous # minor axis of ellipse [mm]
- Size: continuous # 10-log of sum of content of all pixels [in #phot]
- Conc: continuous # ratio of sum of two highest pixels over fSize [ratio]
- Conc1: continuous # ratio of highest pixel over fSize [ratio]
- Asym: continuous # distance from highest pixel to center, projected onto major axis [mm]
- M3Long: continuous # 3rd root of third moment along major axis [mm]
- M3Trans: continuous # 3rd root of third moment along minor axis [mm]
- Alpha: continuous # angle of major axis with vector to origin [deg]
- Dist: continuous # distance from origin to center of ellipse [mm]
- Label: g,h # gamma (signal), hadron (background)

g = gamma (signal): 12332 

h = hadron (background): 6688

In [ ]:
columns = np.array(["Length", "Width", "Size", "Conc", "Conc1", "Asym", "M3Long", "M3Trans", "Alpha", "Dist"])

data = pd.read_csv(pjoin(data_path, "magic.txt"),header=None, names=list(columns)+["Label"])

In [ ]:
X = data[columns].values
y = 1 * (data['Label'].values == "g")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=11, stratify=y)

**Задание**: Проведите краткий EDA, определите, как выглядят признаки, есть ли пропуски итд

In [ ]:
# YOUR CODE

Самый простой способ выбора признаков - Information gain (прирост информации). Прирост информации — это статистика, которая измеряет снижение энтропии (неопределенности) для конкретной функции (таргета) путем деления данных в соответствии с этой характеристикой. Формально, он определен с помощью взаимной информации (I(X , Y) = H(X) – H(X | Y)). Она часто используется в алгоритмах дерева решений, а также имеет другие полезные свойства. Чем выше прирост информации от признака использования признака, тем полезнее он для принятия решений.


In [ ]:
# Apply Information Gain
ig = mutual_info_regression(X_train, y_train)

# Create a dictionary of feature importance scores
feature_scores = {}
for i, col in enumerate(list(columns)):
    feature_scores[col ] = ig[i]

# Sort the features by importance score in descending order
sorted_features = sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)

# Print the feature importance scores and the sorted features
for feature, score in sorted_features:
    print(f"Feature: {feature}, Score: {score}")

In [ ]:
def plot_feature_importances(f_imps, f_names, title=""):
    f_imps = np.array(f_imps)
    f_names = np.array(f_names)
    sort_inds = np.argsort(f_imps)
    yy = np.arange(len(f_imps)).astype(int)
    plt.barh(yy, f_imps[sort_inds])
    plt.yticks(yy, f_names[sort_inds], size=14)
    plt.xticks(size=14)
    plt.xlabel("Feature importance", size=14)
    plt.title(title, size=14)
        
    # Add importance scores as labels on the horizontal bar chart
    for i, v in enumerate(f_imps[sort_inds]):
        plt.text(v + 0.01, i, str(round(v, 3)), color="black", fontweight="bold")
    plt.show()

In [ ]:
plot_feature_importances(list(feature_scores.values()), list(feature_scores.keys()), title="Feature Importance Scores (Information Gain)")

Вопрос: какие из этих признаков можно считать важными? 

Другой вариант - F test. F-Test выполняет проверку гипотез по моделям X и Y, где X — это модель, созданная только с помощью константы, а Y — это модель, созданная с помощью константы и функции.

Ошибки в обеих моделях сравниваются и проверяется, являются ли различия в ошибках между моделями X и Y значительными или они внесены случайно:

Для каждого признака ошибка
$$E[(X[:, i] - mean(X[:, i])) * (y - mean(y))] / (std(X[:, i]) * std(y))$$
преобразуется в F-статистику, после чего рассчитывается уровень значимости признака.

Есть некоторые недостатки использования F-Test для выбора функций. F-Test проверяет и фиксирует только линейные связи между признаками и таргетами. Высококоррелированному признаку присваивается более высокий балл, а менее коррелированному признаку — более низкий балл. Однако даже  сильные нелинейные связи этот тест не выявит.

In [ ]:
f_statistic, p_values = f_classif(X_train, y_train)
feature_scores = {}
feature_scores = {}
for i, col in enumerate(list(columns)):
    feature_scores[col] = (f_statistic[i], p_values[i])

# Sort the features by importance score in descending order
sorted_features = sorted(
    feature_scores.items(), key=lambda x: x[1], reverse=True
    )

# Print the feature importance scores and the sorted features
for feature, score in sorted_features:
    print(f"Feature: {feature}, Score: {score[0]}, p-value: {score[1]}")

**Вопрос**: Какие из столбцов можно считать важными, основываясь на этом тесте? Поясните свой ответ.

**Задание**: 
1) Постройте модель логистической регрессии, используя все признаки из набора.
2) Также постройте модели для вабранных наборов признаков.

In [ ]:
def print_feature_names(features, original_feature_names: list  | None = None):
 if features is not None:
  feature_names = [original_feature_names[int(f.strip("pixel"))] for f in features]
  print(f"Selected features: {feature_names}")
 else:
  print(f"Selected features: {fs.get_feature_names_out()}")

def select_features(X_train, y_train, X_test, function, k: int = 5, features: list  | None = None):
 # configure to select a subset of features
 fs = SelectKBest(score_func=function, k=k)
 fs.fit(X_train, y_train)
 # YOUR CODE: print feature names. Hint: Use the method of fs, which saves out features
 X_train_fs = fs.transform(X_train)
 X_test_fs = fs.transform(X_test)
 return X_train_fs, X_test_fs, fs

In [ ]:
model = DecisionTreeClassifier()
cross_val_score(
    model, X, y, scoring="accuracy", cv=5
).mean()

In [ ]:
results = {"num_features": [], "acc": [], "roc_auc": []}
for i in range(1, len(columns)):
    results["num_features"].append(i)
    # feature selection
    X_train_fs, X_test_fs, fs = select_features(
        X_train, y_train, X_test, f_classif, k=i, features = list(columns)
        )
    # fit the model
    model = DecisionTreeClassifier()
    model.fit(X_train_fs, y_train)
    yhat = model.predict(X_test_fs)
    probs = model.predict_proba(X_test_fs)[:, 1]
    # evaluate predictions
    results["acc"].append(accuracy_score(y_test, yhat))
    results["roc_auc"].append(roc_auc_score(y_test, probs))
results = pd.DataFrame(results)

In [ ]:
results

Мы можем также оценить признаки, используя сам классификатор. В случае с деревьями, лесами и бустингами это очень естественно.

Пусть $T(f)$ все ноды, использующие фичу $f$ для разбиения. Tогда, важность фичи $Imp(f)$ of $f$:
    $$Imp(f) = \sum_{t \in T(f)} n_t \Delta I(t),$$
    $$\Delta I(t) = I(t) - \sum_{c \in children} \frac{n_c}{n_t} I(c),$$
где $n_{t}$ - число объектов в ноде $t$; $I(t)$ – impurity function (gini, cross-entropy, MSE) в ноде $t$
  
В случае ансамбля важности фичей усредняются между деревьями.

In [ ]:
model.fit(X_train, y_train)
plot_feature_importances(model.feature_importances_, columns, title="Feature Importance Scores (Information Gain)")

В случае линейной модели, важность признака тоже можно оценить. В случае отнормализованных признаков, оценкой важности выступает модель соответствующего коэффициента.

**Задание**: Оцените важность признаков, используя линейную модель. 
Подсказка: используйте l2 регуляризацию и <model>.coef_[0] для получения коэффициентов.

In [ ]:
def get_feature_imp_with_lin_mod(X_train, y_train, C=1.0):
    ss = StandardScaler()
    ss.fit(X_train)
    X_train_ss = ss.transform(X_train)
    ### YOUR CODE
    #
    ### YOUR CODE
    return f_imps_lin

Как видно, для разных моделей оценки сильно отличаются. 
Общий метод оценки важности: 
1) Обучите свою модель
2) Рассчитайте метрику качества  $Q_o$  на тестовом наборе
3) Для признака $f$:
     1) Замените значения случайными значениями из того же распределения (выполните случайное перемешивание)
     2) Рассчитайте метрику качества $Q_r$ на тестовом наборе 
     3) Оцените важность признака: $Imp(f) = Q_0 - Q_r$

**Задание**: Реализуйте такой метод, используя линейную модель с регуляризацией.

**Бонусное задание**: Сделайте метрику аргументом, проведите тесты с линейной моделью, деревом решений, случайным лесом, двумя метриками. Постройте таблицу со сравнением.

In [ ]:
def get_feature_imp_general(X_test, y_test, model):
    feature_importances = []
    y_test_proba = model.predict_proba(X_test)[:, 1]
    
    q_0 = accuracy_score(y_test, y_test_proba)
    for i in range(X_test.shape[1]):

        # do not forget to make a copy of X_test!
        X_test_copy = X_test.copy()

        # shuffle values of the i-th feature
        ### YOUR CODE
        #
        ### YOUR CODE
        
        X_test_copy = np.nan_to_num(X_test_copy)
        y_hat = model.predict(X_test_copy)
        q_r = accuracy_score(y_test, y_hat)

        importance = None # !! your code (calculate importance according to algorithm)
        feature_importances.append(importance)

    return np.array(f_imps_gen)

Как же перейти от простой оценки важности признаков к поиску такого подмножества, которое приведет к наибольшей точности модели?
Можно использовать рекурсиввный выбор признаков (Recursive feature selection, или RFE).
RFE работает путем поиска подмножества признаков, начиная со всех признаков в наборе и постепенно удаляя самые незначимые, пока не останется желаемое количество.

Это достигается путем подбора модели для оценки, ранжирования признаков по важности, отбрасывания наименее важного признака и повторного обучения модели. Этот процесс повторяется до тех пор, пока не останется заданное количество признаков. Характеристики оцениваются либо с использованием предоставленной модели, либо с использованием статистического метода, такого как f-score или IG.

В sklearn реализован класс трансформера для такого подбора, при этом подюор может быть отделен от основной модели. 

In [ ]:
# scaler = StandardScaler()
rfe = RFE(estimator=RandomForestRegressor(), n_features_to_select=5)
model = LogisticRegression()
#pipeline = Pipeline(steps=[('sc',scaler),('s',rfe),('m',model)])
pipeline = Pipeline(steps=[('s',rfe),('m',model)])
# evaluate model
pipeline.fit(X_train, y_train)
print_feature_names(rfe.get_feature_names_out(), columns)
# evaluate the model
y_hat = pipeline.predict(X_test)
# evaluate predictions
acc = accuracy_score(y_test, y_hat)
print('Accuracy: %.3f' % acc)

**Задание**: Реализуйте данный метод вручную. Бонусное задание: реализуйте класс трансформера для подбора и/или конфигурируемые метрику и модель.

In [ ]:
def find_best_features_set(X_train, y_train, X_test, y_test, model, columns):
    """
    Recursive feature elimination
    """
    X_train_curr = X_train.copy()
    X_test_curr  = X_test.copy()
    f_names_curr = columns.copy()

    # define a list for the feature importances
    scores = []
    # for each feature in the sample estimate its importance
    for i in range(X_test.shape[1]):
        # fit the model using current set of festures
        # get feature importances
        ### YOUR CODE
        #
        # importances = ...
        ### YOUR CODE
        yhat = model.predict(X_test_curr)
        score = accuracy_score(y_test, y_hat)
        print("Score: ", np.round(score, 4))
        scores.append(score)
        # remove feature with the least importance
        min_importance = importances.min()
        X_train_curr = X_train_curr[:, importances > min_importance]
        X_test_curr  = X_test_curr[:, importances > min_importance]
        f_names_curr = f_names_curr[importances > min_importance]
    return scores, f_names_curr

In [ ]:
scores, f_names  = find_best_features_set(X_train, y_train, X_test, y_test, model, columns)
nf = np.arange(1, len(scores)+1)[::-1]
plt.figure(figsize=(9, 6))

plt.plot(nf, scores, linewidth=3)
plt.scatter(nf, scores, linewidth=3)

plt.xlabel("Number of features", size=16)
plt.ylabel("Score", size=16)
plt.xticks(size=16)
plt.yticks(size=16)
plt.grid()
plt.show()

### PCA

[Ноутбук Евгения Соколова](https://github.com/esokolov/ml-course-hse/blob/master/2022-spring/seminars/sem14_pca_tsne.ipynb)

Выделение новых признаков путем их отбора часто дает плохие результаты, и в некоторых ситуациях такой подход практически бесполезен. Например, если мы работаем с изображениями, у которых признаками являются яркости пикселей,
невозможно выбрать небольшой поднабор пикселей, который дает хорошую информацию о содержимом картинки. Поэтому признаки нужно как-то комбинировать.\n",

__Метод главных компонент__ &mdash; один из самых интуитивно простых и часто используемых методов для снижения размерности данных и проекции их на ортогональное подпространство признаков. В рамках метода делается два важных упрощения задачи:
1. игнорируется целевая переменная;
2. строится линейная комбинация признаков.
П. 1 на первый взгляд кажется довольно странным, но на практике обычно не является таким уж плохим. Это связано с тем, что часто данные устроены так, что имеют какую-то внутреннюю структуру в пространстве меньшей размерности, которая никак не связана с целевой переменной. Поэтому и оптимальные признаки можно строить не глядя на ответ.
П. 2 тоже сильно упрощает задачу, но далее мы научимся избавляться от него.

PCA ищет линейную комбинацию переменных, чтобы мы могли извлечь максимальную дисперсию из них. По завершении этого процесса он удаляет ее и ищет другую линейную комбинацию (фактор), которая дает объяснение максимальной пропорции оставшейся дисперсии, что приводит к ортогональным факторам. Это повторяется столько раз, чтобы достигнуть целевого числа объясненной дисперсии или числа факторов.

### Теория
Кратко вспомним, что делает этот метод.
Пусть $X$ &mdash; матрица объекты-признаки, с нулевым средним каждого признака, а $w$ &mdash; некоторый единичный вектор. Тогда $Xw$ задает величину проекций всех объектов на этот вектор. Далее ищется вектор, который дает наибольшую дисперсию полученных проекций (то есть наибольшую дисперсию вдоль этого направления):
$$\max_{w: \|w\|=1} \| Xw \|^2 =  \max_{w: \|w\|=1} w^T X^T X w$$
Подходящий вектор тогда равен собственному вектору матрицы $X^T X$ с наибольшим собственным значением. После этого все пространство проецируется на ортогональное дополнение к вектору $w$ и процесс повторяется.
   

In [ ]:
data_synth = np.random.multivariate_normal(
    mean=[0, 0],
    cov=[[4, 0],
         [0, 1]],
    size=1000
)

Теперь изобразим точки выборки на плоскости и применим к ним PCA для нахождения главных компонент. В результате работы PCA из sklearn в dec.components_ будут лежать главные направления (нормированные), а в dec.explained_variance_ — дисперсия, которую объясняет каждая компонента. Изобразим на нашем графике эти направления, умножив их на дисперсию для наглядного отображения их значимости.

In [ ]:
from sklearn.decomposition import PCA

def PCA_show(dataset):
    plt.scatter(*zip(*dataset), alpha=0.5)

    dec = PCA()
    dec.fit(dataset)
    ax = plt.gca()
    for comp_ind in range(dec.components_.shape[0]):
        component = dec.components_[comp_ind, :]
        var = dec.explained_variance_[comp_ind]
        start, end = dec.mean_, component * var
        ax.arrow(start[0], start[1], end[0], end[1],
                 head_width=0.2, head_length=0.4, fc='r', ec='r')

    ax.set_aspect('equal', adjustable='box')

plt.figure(figsize=(16, 8))
PCA_show(data_synth)

Видим, что PCA все правильно нашел. Но это, конечно, можно было сделать и просто посчитав дисперсию каждого признака. Повернем наши данные на некоторый фиксированный угол и проверим, что для PCA это ничего не изменит.

In [ ]:
angle = np.pi / 6
rotate = np.array([
        [np.cos(angle), - np.sin(angle)],
        [np.sin(angle), np.cos(angle)],
    ])
data_synth_2 = rotate.dot(data_synth.T).T

plt.figure(figsize=(16, 8))
PCA_show(data_synth_2)

Ниже пара примеров, где PCA отработал не так хорошо (в том смысле, что направления задают не очень хорошие признаки).

Вопрос: почему так произошло?

In [ ]:
np.random.seed(123) # поиграйте с сидами))

data_synth_bad = [
    make_circles(n_samples=1000, factor=0.3, noise=0.1)[0]*2,
    make_moons(n_samples=1000, noise=0.15)[0]*2,
    make_blobs(n_samples=1000, n_features=2, centers=4)[0]/5,
    np.random.multivariate_normal(
        mean=[0, 1.5],
        cov=[[2, 1],
             [1, 1]],
        size=1000),
]

plt.figure(figsize=(16,8))
rows, cols = 2, 2
for i, data in enumerate(data_synth_bad):
    plt.subplot(rows, cols, i + 1)
    PCA_show(data)

In [ ]:
scaler = StandardScaler()


Доп.задание : Запустите предыдущий метод с учетом скейлинга. Поменяется ли результат?

In [ ]:
pca = PCA(n_components = 5)
pipeline = Pipeline(steps=[('sc', scaler), ('pca',pca),('m',model)])
# evaluate model
pipeline.fit(X_train, y_train)
pcas = " ".join(f"{i:.2}" for i in pca.explained_variance_ratio_)
print(f"Explained {pcas} ({pca.explained_variance_ratio_.sum():.2f}) ratio of variance")
# evaluate the model
y_hat = pipeline.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, y_hat)
print('MAE: %.3f' % mae)

Почему важно использование скейлинга перед проведением PCA?
Если один объект варьируется больше, чем другие, только из-за их соответствующих масштабов, PCA определит, что такой объект доминирует в направлении главных компонентов.

In [ ]:
pca = PCA(n_components = 5)
pipeline = Pipeline(steps=[('pca',pca),('m',model)])
# evaluate model
pipeline.fit(X_train, y_train)
pcas = " ".join(f"{i:.2}" for i in pca.explained_variance_ratio_)
print(f"Explained {pcas} ({pca.explained_variance_ratio_.sum():.2f}) ratio of variance")
# evaluate the model
y_hat = pipeline.predict(X_test)
# evaluate predictions
mae = mean_absolute_error(y_test, y_hat)
print('MAE: %.3f' % mae)

 Какой метод наиболее страдает от проклятия размерности?
 KNN.

 Покажем это.

Рассмотрим набор MNIST - набор рукописных чисел от 0 до 9. Это один из классических наборов для компьютернорго зроения, состоящий из 100 тысяч черно-белых изображений 28х28. Однако матрицы можно и вытянуть в вектор, чтобы предсказывать класс в табличном формате. Тогда мы получим пространство признаков размера 784!

In [ ]:
mnist = fetch_openml('mnist_784', cache=False)

X = mnist["data"].astype('float64')
X.reset_index()
y = mnist["target"].astype('int64')
y.reset_index()

print("\nNo. of Samples: ", X.shape)
print("No. of Labels: ", y.shape)

Так выглядит набор данных в табличном виде

In [ ]:
X.head()

Рассмотрим случайный семпл в данных (преобразовав обратно в матрицу)

In [ ]:
random_digit = X.loc[10,:].values

random_digit_image = random_digit.reshape(28, 28)

plt.imshow(random_digit_image, cmap = 'gray', interpolation="nearest")
#plt.axis("off")
plt.show()


Отнормируем

In [ ]:
X /= 255.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

Заодно с обучением модели вспомним о поиске гиперпараметров. Что это такое?

Так как данных очень много, равномерно засемплим 1000 точек. В общем случае это плохая практика. Почему?

In [ ]:
random_indices = np.random.choice(X.index, 10000, replace=False)
X_rus = X.loc[random_indices]
y_rus = y.loc[random_indices]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus, test_size=0.2, random_state=0)
print(X_train.shape, y_test.shape)

In [ ]:
param_grid = {'n_neighbors': [3, 4], 'p': [2], 'weights': ["distance"]}


knn_clf = KNeighborsClassifier()

knn_cv = GridSearchCV(knn_clf, param_grid, scoring='f1_micro', cv=5, verbose=1, n_jobs=-1)
knn_cv.fit(X_train, y_train)


params_optimal_knn = # YOUR CODE: get best params
print("Best Score: %f" % knn_cv.best_score_)
print("Optimal Hyperparameter Values: ", params_optimal_knn)
print("\n")

In [ ]:
%%time
##########
# YOUR CODE 
# fit knn with best params
#########
y_train_predicted = knn.predict(X_train)

train_accuracy_knn = np.mean(y_train_predicted == y_train)
print("\nTraining Accuracy: ", train_accuracy_knn)

In [ ]:
%%time

y_train_pred = cross_val_predict(knn, X_train, y_train, cv=5)
confusion_matrix(y_train, y_train_pred)

In [ ]:

y_test_predicted = knn.predict(X_test)

test_accuracy_knn = np.mean(y_test_predicted == y_test)
print("\nTraining Accuracy: ", test_accuracy_knn)

In [ ]:
%%time

# The accuracy of the model
test_accuracy_knn = knn.score(X_test, y_test)
print("\nTest Accuracy: ", test_accuracy_knn)


# No. of Correct Predictions
y_test_predicted = knn.predict(X_test)
print("\nNo. of correct predictions (Test): %d/%d" % (np.sum(y_test_predicted == y_test), len(y_test)))


# Confusion Matrix
print("\nConfusion Matrix (Test Data):\n", confusion_matrix(y_test, y_test_predicted))

In [ ]:
pd.crosstab(y_test, y_test_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Попробуем уменьшить влияние проклятия размерности с помощью уменьшения размерности, использовав метод PCA.

In [ ]:
%%time
pca = PCA(n_components=0.95)
pca.fit(X_train)

print("Number of Principle Components: ", pca.n_components_)

In [ ]:
#################
# YOUR CODE
# Transform x train and test
# X_train_pca = 
# X_test_pca =
###################

In [ ]:
%%time


knn_pca = KNeighborsClassifier(n_neighbors=4, p=2, weights="distance")

knn_pca.fit(X_train_pca, y_train)


y_test_predicted_pca_knn = knn_pca.predict(X_test_pca)
print("KNN (PCA): Test Accuracy: ", accuracy_score(y_test, y_test_predicted_pca_knn))

print("\nKNN (PCA): Test Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted_pca_knn))

print("\nKNN (PCA): Classification Report:")
print(classification_report(y_test, y_test_predicted_pca_knn))

print("\n")

Мы видим, что после уменьшения размерности мы можем снизить стоимость расчета расстояния в K-NN, что приводит к значительному сокращению времени обучения. При этом мы не потеряли в точности, а даже выиграли!

Для сравнения времени обучения рассмотрим метод, построенный не на расстоянии - случайный лес. Эта модель будет страдать от проклятия размерности в терминах времени меньше.

In [ ]:
%%time

forest_clf = RandomForestClassifier(n_estimators=1000, criterion="gini", 
                                    max_depth=32, class_weight="balanced", oob_score=True, verbose=1, n_jobs=-1)

forest_clf.fit(X_train, y_train)

y_test_predicted_rf = forest_clf.predict(X_test)
print("Random Forest: Test Accuracy: ", accuracy_score(y_test, y_test_predicted_rf))

print("\nRandom Forest: Test Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted_rf))

print("\nRandom Forest: Classification Report:")
print(classification_report(y_test, y_test_predicted_rf))

print("\n")

In [ ]:
%%time

forest_clf = RandomForestClassifier(n_estimators=1000, criterion="gini", 
                                    max_depth=32, class_weight="balanced", oob_score=True, verbose=1, n_jobs=-1)

forest_clf.fit(X_train_pca, y_train)

y_test_predicted_rf = forest_clf.predict(X_test_pca)
print("Random Forest: Test Accuracy: ", accuracy_score(y_test, y_test_predicted_rf))

print("\nRandom Forest: Test Confusion Matrix:")
print(confusion_matrix(y_test, y_test_predicted_rf))

print("\nRandom Forest: Classification Report:")
print(classification_report(y_test, y_test_predicted_rf))

print("\n")

Как видно, обучение произошло гораздо быстрее, при этом точность алгоритма даже выше, чем у KNN

## Проиллюстрируем

In [ ]:
def process_image(filename, size=64):
    with Image.open(filename) as img:
        # Convert to grayscale and resize
        img = img.convert('L').resize((size, size))
        
        # Convert to numpy array with channel dimension
        arr = np.array(img, dtype=np.uint8)[..., np.newaxis]
        return arr

Создадим небольшой датасет с картинками котов. Для простоты переведем ихз в черно-белую гамму и сделаем их размера 64 на 64. ПОсмотрим, что получится. При этом выведем еще и "среднего кота". Потом нам это понадобится для PCA.

In [ ]:
from PIL import Image
import random
random.seed(1234)
cats_path = os.path.join(data_path, "base_cats")
image_shape = (64, 64)
rows, cols = 2, 4
n_samples = rows * cols

all_files = [f for f in os.listdir(cats_path ) 
                if f.lower().endswith(('.jpg', '.jpeg'))]
processed = []
for filename in random.sample(all_files, 120):
    processed.append(process_image(os.path.join(cats_path, filename)))
    
sample = random.sample(processed, n_samples)
cats_data = np.array(processed, dtype=np.float32)

mean_cat = cats_data.mean(axis=0)

plt.figure(figsize=(16, 8))

for i in range(n_samples - 1):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(cats_data[i, :].reshape(image_shape), interpolation='none',
               cmap='gray')
    plt.xticks(())
    plt.yticks(())
    
plt.subplot(rows, cols, n_samples)
plt.imshow(mean_cat.reshape(image_shape), interpolation='none',
           cmap='gray')
plt.xticks(())
_ = plt.yticks(())

Применим к ним PCA. Найдем первые главные компоненты

In [ ]:
model_pca = PCA()
cats_data -= mean_cat  # отнормировали данные к нулевому среднему
model_pca.fit(cats_data.reshape(-1, 64*64))

plt.figure(figsize=(16, 8))
rows, cols = 2, 4
n_samples = rows * cols
for i in range(n_samples):
    plt.subplot(rows, cols, i + 1)
    plt.imshow(model_pca.components_[i, :].reshape(image_shape), interpolation='none', cmap='gray')
    plt.xticks(())
    plt.yticks(())

Получились забавные призраки! Обратите внимание, что это не изображение котов, а именно компоненты - или собственные векторы $X^T X$.